<a href="https://colab.research.google.com/github/iwatake2222/colaboratory_study/blob/master/DL_tutorial/DL_tutorial_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learningアプリケーション開発 (1) Keras with Python
Kerasで簡単にMNIST数字識別モデルを作り、Pythonで確認

https://qiita.com/iwatake2222/items/796ec8560563625ace34

# 環境設定

In [0]:
from google.colab import drive 
drive.mount("/content/drive")
!ls

# Kerasモデル作成、学習、h5モデル保存

In [0]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np

print(tf.__version__)

# MNISTの学習用データ、テストデータをロードする
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# 学習データの前処理
# X: 6000x28x28x1のTensorに変換し、値を0～1.0に正規化
# Y: one-hot化(6000x1 -> 6000x10)   -> no need when using SparseCategoricalCrossentropy()
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_train = x_train / 255.
x_test = x_test / 255.
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
# y_train = tf.keras.utils.to_categorical(y_train, 10)
# y_test = tf.keras.utils.to_categorical(y_test, 10)

# 学習状態把握用のTensorBoard設定
tsb = tf.keras.callbacks.TensorBoard(log_dir="./logs")

# Convolutionモデルの作成
input = tf.keras.layers.Input(shape=(28,28,1), name="input_image")
conv1 = tf.keras.layers.Conv2D(
  filters=8,
  kernel_size=(3,3),
  strides=(1,1),
  padding="same",
  activation="relu"
)(input)
pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(conv1)
conv2 = tf.keras.layers.Conv2D(
  filters=4,
  kernel_size=(3,3),
  strides=(1,1),
  padding="same",
  activation="relu"
)(pool1)
dropout1 = tf.keras.layers.Dropout(0.2)(conv2)
flatten1 = tf.keras.layers.Flatten()(dropout1)
output = tf.keras.layers.Dense(units=10, activation="softmax", name="output_scores")(flatten1)
model = tf.keras.models.Model(inputs=[input], outputs=[output], name="ConvMnist")
model.summary()

model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

# Convolutionモデルの学習
model.fit(
  x_train,
  y_train,
  batch_size=32,
  epochs=20,
  validation_split=0.2,
  callbacks=[tsb],
)

# 学習したモデルを使用して、テスト用データで評価する
model.evaluate(x_test,  y_test, verbose=2)

# 学習済みモデルをファイル(h5)に保存する
model.save("conv_mnist.h5")

# 推論テスト

In [0]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

# Read input image
!wget -O 4.jpg  "https://drive.google.com/uc?export=download&id=1-3yb3qCrN8M6Bdj7ZZ9UMjONh34R2W_A" 
img_org = cv2.imread("4.jpg")
cv2_imshow(img_org)

# Pre process
## グレースケール化、リサイズ、白黒判定、価範囲を0～255 -> 0.0～1.0
img = cv2.cvtColor(img_org, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (28, 28))
img = 255 - img
img = img / 255.
img = img.astype(np.float32)
input_tensor = img.reshape(1, img.shape[0], img.shape[1], 1)

# Load model
loaded_model = tf.keras.models.load_model("conv_mnist.h5")

# Inference
scores = loaded_model.predict(input_tensor)

result = np.argmax(scores[0])
print("predicted number is {} [{:.2f}]".format(result, scores[0][result]))

In [0]:
# Save to google drive
!cp conv_mnist.h5  "/content/drive/My Drive/test_data/number"

# Download to local
from google.colab import files
files.download( "./conv_mnist.h5")